In [ ]:
import soundfile as sf
import matplotlib.pyplot as plt
import numpy as np
import sounddevice as sd
import math
from scipy.io import wavfile
import wave
import contextlib
from scipy.fft import fft,ifft
from IPython.display import display, Audio

# Preparation

Soit x[n] un signal audio (réel) discret et xmat sa matrice de TFCT, de dimensions M × L
définies au TP2. xmat est calculée à partir des paramètres Nwin, Nhop et Nf f t également définis
au TP2.

1. Quel est l’intérêt de choisir Nwin pair pour le calcul de xmat ?
    the interest
    
2. Quel est l’intérêt de choisir Nf f t ≥ Nwin pour le calcul de xmat ?

    before starting the study of the matrix, I am interested in taking a pair Nwin

The interest of getting an nfft greater than nwin is that, with this, we will have a greater density of points as a result of the fourier transform for that trame. In this way, we can understand more clearly which frequencies are most present in our signal. Normally, we have frequency peaks between those that we have with the discrete signal, causing small distortions when finding the most influential frequencies. The smaller the number of points as a result of our fft, and consequently the smaller our nfft, the greater this distortion. In bref, the nfft affects the frequency signal accuracy.

3. Quel est l’intérêt de choisir Nf f t = Nwin (et pas Nf f t > Nwin) pour le calcul de xmat ?

choosing an nfft greater than n win means adding zeros to the end of the windowed signal. This addition can mean changing the original sign since we add information to the end of the sign (zeros) that is not necessarily the real one. This change in the signal can be translated into the fft so that false frequency peaks appear in the windowed signal. The greater the difference between Nfft and Nwin, the more evident these secondary peaks become, which can lead to erroneous interpretations. Therefore, sometimes it is preferable not to perform zero padding on the windowed signal.

4. Montrer que la TFD correspondant à la lème trame fenêtrée de x[n] s’écrit :
$Xl[1], .., Xl[M], Xl[M − 1]*, .., Xl[2]*$
où Xl[k]* désigne le conjugué de Xl[k].

$X[M] = \sum _{i=1} ^{N-1} = x[N]* e^{\frac{−j2πM*n}{N}}$

$X[M-N] = \sum _{i=1} ^{N-1} = x[N]* e^{\frac{−j2π(M-N)*n}{N}}$

$X[M-N] = \sum _{i=1} ^{M-1} = x[n]* e^{−j2π*n + \frac{j2πM*n}{N}}$

$X[M-N] = \sum _{i=1} ^{M-1} = x[n]* e^{ \frac{j2πM*n}{N}} \times e^{−j2π*n} $

but knowing that 

$e^{−j2πn} = (e^{−jπ})^{2n}$

and

$e^{−jπ} = 1$

we have, then:

$X[M-N] = \sum _{i=1} ^{M-1} x[n]* e^{ \frac{j2πM*n}{N}} $

by euler's equation, we can see that when we inverse the signal of the angle, we have the conjugate of the original number, so we can assume that:

$e^{ \frac{j2πM*n}{N}} = (e^{ - \frac{j2πM*n}{N}})^{*}$

and this leads us to:

$x[N]* e^{ \frac{j2πM*n}{N}} = (x[n]* e^{ \frac{j2πM*n}{N}})^*$

$X[N - M] = (X[N])^*$

and this proves that:

X[1] = [M-1], X[2] = [M-2], ... 

with it, we can see that the FFT has a symmetry around M/2. This proves that we can evaluate the signal halfway and from there just repeat what we found so far, but in reverse order.

5. Exprimer la TFD inverse de (Xl[k]) k ∈ [1..Nfft], notée (xl[n]) n ∈ [1..Nwin].

The formula for that is:

${x[n] = {\frac {1}{Nfft}}\sum _{k=1}^{Nfft-1}X_{k}\cdot e^{i{\frac {2\pi }{Nfft}}kn}}$  

where n ∈ [1..Nwin].

6. A quelle(s) condition(s) sur (Xl[k])k∈[1..Nfft] la TFD inverse (xl[n]) n∈[1..Nwin] est-elle réelle ?

# Implémentation de la TFCT inverse par Overlap-Add (OLA)

Our objective here will be to use the function created in the last TP, which performs the Short time fourier tramsform, to see if we can reverse the process and get back the original sound signal from it.
For this we will use the Overlap-Add (OLA) algorithm.

The algorithm consists of reconstituting each segment of the Matrix obtained from the STFT.

After that, we move each reconstituted chunck so that it is in the right position, according to the original signal.

Then we add all these chuncks.

And finally, we normalize, taking into account the window function used in STFT (hamming function)

First, we take the zeropad and TFCT functions created in the last TP

In [ ]:
#From tp2
def zeropad(array, length):
    
    len_dif = length - len(array)
    zero_array = np.zeros(len_dif)

    return np.concatenate((array, zero_array))

In [ ]:
#From tp2
def tfct(document, N_win, N_hop, N_fft):
    xvect = sf.read(document, dtype='float64')
    xvect = xvect[0]
    
    with contextlib.closing(wave.open(document,'r')) as f:
        rate = f.getframerate()

    x_length = len(xvect)
    number_of_ffts = int(x_length/N_hop)

    x_mat = np.zeros((N_fft,number_of_ffts))

    for i in range(number_of_ffts):

        xvect_trame = xvect[i*N_hop:i*N_hop + N_win]

        windowed_xvect = [x*hamming_win[i] for i, x in enumerate(xvect_trame)]

        zero_paddedwindowed_xvect = zeropad(windowed_xvect, N_fft)

        fft_zeropadded_windowed_xvect = fft(zero_paddedwindowed_xvect)

        x_mat[:,i-1] = np.transpose(np.abs(fft_zeropadded_windowed_xvect))
                
    t = np.linspace(0, len(xvect)/rate, number_of_ffts)
    f = np.linspace(0, rate, N_fft)
    
    return t, f, x_mat 

In [ ]:
#from tp2

def extents(f):
  delta = f[1] - f[0]
  return [f[0] - delta/2, f[-1] + delta/2]

In [ ]:
def tfct(document, N_win, N_hop, N_fft):
    xvect = sf.read(document, dtype='float64')
    xvect = xvect[0]
    
    with contextlib.closing(wave.open(document,'r')) as f:
        rate = f.getframerate()

    x_length = len(xvect)
    number_of_ffts = int(x_length/N_hop)

    x_mat = np.zeros((N_fft,number_of_ffts))
    hamming_win = np.hamming(N_win)

    for i in range(number_of_ffts):

        xvect_trame = xvect[i*N_hop:i*N_hop + N_win]

        windowed_xvect = [x*hamming_win[i] for i, x in enumerate(xvect_trame)]

        zero_paddedwindowed_xvect = zeropad(windowed_xvect, N_fft)

        fft_zeropadded_windowed_xvect = fft(zero_paddedwindowed_xvect)

        x_mat[:,i-1] = np.transpose(np.abs(fft_zeropadded_windowed_xvect))
                
    t = np.linspace(0, len(xvect)/rate, number_of_ffts)
    f = np.linspace(0, rate, N_fft)
    
    return t, f, x_mat

Now let's go to the first step of the OLA Algorithm: reconstituting each transformed piece.

These chunks correspond to the columns of the STFT matrix.
To constitute them, we will use the ifft function, from the Numpy.fft sub-library.

This function transforms a frequency signal back into the time domain. As an imput we give a column of the matrix and as a result we have a time signal of the same size.

In [ ]:
def reconstruct_trame(stft_matrix):    
    for i, trame in enumerate(stft_matrix):
        reconstructed_list = abs(ifft(trame))

Now we need to somehow shift these pieces to, then add them up and get a vector of the same size as the original sound signal.

The shortest way will be to create an array of zeros with the correct size.

The correct dimension of this vector will be the number of chunks (number of columns) times the STFT advance step (nHop). The problem is that for the last iteration, we must add a vector of the window size (nWin) and not the step size (nHop). So we must add the difference between them (nWin - nHop)

In [ ]:
nWin = 1024
nHop = round(N_win/2)
number_of_columns = 500


np.zeros(int(nHop*number_of_columns + abs(nWin - nHop)))


NameError: name 'N_win' is not defined

With this array of zeros, instead of shifting the reconstituted chuncks, we can shift the time vector itself to add the reconstituted chuncks to it with this line of code:

OLA_array[int(i*nHop) : int(i*nHop + nWin)] += reconstructed_list


Our function so far looks like this:

In [ ]:
def itfct(stft_matrix, nHop, nWin):
    
    [_, L] = np.shape(stft_matrix)
    
    OLA_array = np.zeros(int(nHop*L + abs(nWin - nHop)))
    
    for i, trame in enumerate(np.transpose(stft_matrix)):
        reconstructed_list = abs(ifft(trame))

        OLA_array[int(i*nHop) : int(i*nHop + nWin)] += reconstructed_list
    
    return OLA_array

The last step is to normalize the obtained array.
For this we must take into account the window function used in sftf. In our case this function is the hamming function.

As seen in class, we know that the reconstituted chuncks, when superimposed in the OLA algorithm, end up with wrong values thanks to this superposition. To fix this distortion in the values we just have to divide each value of our vector obtained with the itfct by the sum of the values of the window function sampled in the same way as the sound signal divided by nHop.

This is done by the following line of code:

K = sum(np.hamming(nWin))/nHop
    
OLA_array /= K

We must also add as an output element of our function, the temporal array, which will help us to plot the signal obtained and to orient ourselves temporally.

For this we must know a priori the sampling frequency of the desired sound signal.

Ours should look like this now:

In [ ]:
## Now with normalization

def itfct(stft_matrix, nHop, nWin, sf):
    
    [_, L] = np.shape(stft_matrix)
    
    final_reconstructed = np.zeros(np.shape(stft_matrix))
    OLA_array = np.zeros(int(nHop*L + abs(nWin - nHop)))
    
    for i, trame in enumerate(np.transpose(stft_matrix)):
#         reconstructed_list = np.real(ifft(trame))[0 : nWin]
        reconstructed_list = abs(ifft(trame))[0 : nWin]

        OLA_array[int(i*nHop) : int(i*nHop + nWin)] += reconstructed_list
    
    K = sum(np.hamming(nWin))/nHop
    
    OLA_array /= K
    
    # creating time array
    t = np.linspace(0, len(OLA_array)/sf, len(OLA_array)) 
    
    return t, OLA_array

Now we need to test our function.
For this, first we use the 'sound.wav' file, perform the STFT on it and plot the resulting matrix as a spectgram
The parameters to be provided are:

Nwin : The value will be the size of the original signal divided by 500.

Nhop: It will be the value of Nwin divided by 2. This way we will have 1000 windowed pieces of the original signal.

Nfft: Twice the value of Nwin, this way we improve the frequency accuracy obtained, through zero padding of the windowed signal.

F : Sampling frequency. I used the value of 16 kHz.

In [ ]:
document = 'sound.wav'

# read signal
x_array = sf.read(document, dtype='float64')
x_array = x_array[0]
f = 16000

# Initializate parameters
x_length = len(x_array)
N_win = round(x_length/500)
N_hop = round(N_win/2)
N_fft = N_win*2

# get stft matrix
t, f, matrix = tfct(document,N_win, N_hop, N_fft)

import seaborn as sns

plt.imshow(np.log10(matrix), aspect='auto', interpolation='none',
 extent = extents(t) + extents(f), origin='lower')

plt.xlabel('t (s)')
plt.ylabel('f (Hz)')
plt.show()

After that, we use this matrix as input for the function developed today, along with the other reconstruction parameters (nWin, nHop and sf)

In [ ]:
t_y, y_array = itfct(matrix, N_hop, N_win, 16000)

We can plot the obtained vector overlapping with the original vector to see the differences

In [ ]:
document = 'sound.wav'
x_array = sf.read(document, dtype='float64')
x_array = x_array[0]
f = 16000

t_x = np.linspace(0, len(x_array)/f, len(x_array)) 

plt.plot(t_x,x_array, label = "original signal")

plt.plot(t_y, y_array, label = "reconstructed signal")

print("play origial sound:")
display (Audio(x_array , rate=f))
print("play recovered sound:")
display (Audio(y_array , rate=f))

plt.legend()

We can see that the values of the reconstructed signal were all positive. I tried to review the steps and unfortunately I couldn't understand the explanation for this.

Nevertheless, we can see the similarity of both signals.

When we listen to the sound produced by the signal obtained, we notice a difference between them. However, we can clearly recognize that the song is the same.

Now we need to study the error obtained by the transformation

signal -> stft -> stft inverse -> rebuilt signal

for this we look at the mean square of the difference between them.

we can also plot the error.

Unfortunately, the expected is to have a big error, since the signal came entirely positive.

In [ ]:
error = x_array - y_array[:len(x_array)]

plt.plot(error)

squared_error = np.average(np.sqrt(error**2))

print("squared error:", squared_error)

# Application au débruitage pour soustraction spectrale

Based on the article:

Steven F. Boll (1979). Suppression of Acoustic Noise in Speech Using Spectral Subtraction. IEEE Transactions on Acoustics, Speech, and Signal Processing,
vol.27(2), p.113-120.

For this part, we will use the noise suppression technique.

First, let's test what we've produced so far, but with the mix.wav file

We will use the parameters in the same way as before. Nwin = signal size/500 and Nhop = Nwin/2

In [ ]:
document = 'mix.wav'

# read signal
x_array = sf.read(document, dtype='float64')
x_array = x_array[0]
f = 16000

# Initializate parameters
x_length = len(x_array)
N_win = round(x_length/500)
N_hop = round(N_win/2)
N_fft = N_win*2

# get stft matrix
t, f, matrix = tfct(document,N_win, N_hop, N_fft)

import seaborn as sns
plt.imshow(np.log10(matrix), aspect='auto', interpolation='none',
 extent = extents(t) + extents(f), origin='lower')

plt.title("Mix sound spectogram")
plt.xlabel('t (s)')
plt.ylabel('f (Hz)')
plt.show()

Now, to eliminate the noise, we need to understand the characteristics of this noise.

Let's try to find a piece of the signal where we can only hear the noise

In [ ]:
print(x_array)
f = 16000

t_x = np.linspace(0, len(x_array)/f, len(x_array))

plt.plot(t_x[:7500], x_array[:7500])
plt.xlabel('t (s)')

We can notice that from the beginning until the instant 0.46 s we have nothing but noise.
Now, let's take this piece of noise and study what frequencies it is composed of.

In [ ]:
noise_signal = x_array[:7500]

noise_fft = fft(noise_signal)

plt.figure()
plt.plot(np.abs(noise_fft)/len(noise_fft))
plt.title("noise frequence spectrum")
plt.xlabel("Frequencies (Hz)")
plt.grid()

Now, we must take this frequency information from the noise and subtract it from each leg (column) of the matrix returned by the tfct() function.

For this we have to adjust the parameters so that the noise frequency array has the same dimension as the matrix columns

The second 0.46, corresponding to the instant up to which there is only noise, is translated by the first 7500 values of the original signal. Knowing this, we need that the nWin of our STFT is also equal to 7500.

First, let's see how the array looks with the parameters described like this. Without the indicated subtraction.

In [ ]:
# Initializate parameters
x_length = len(x_array)
N_win = 7500
N_hop = round(N_win/2)
N_fft = N_win

# get stft matrix
t, f, matrix = tfct(document,N_win, N_hop, N_fft)       

import seaborn as sns
plt.imshow(np.log10(matrix), aspect='auto', interpolation='none',
 extent = extents(t) + extents(f), origin='lower')

plt.title("Mix sound spectogram")
plt.xlabel('t (s)')
plt.ylabel('f (Hz)')
plt.show()     

Now, we perform the subtractions and see how the matrix changes.

It is important to note that there are no negative amplitudes for a sine wave. So when performing the subtraction we must check if the result is negative. If yes, it must be changed to 0.

This is done through the maximo function:
max(result of subtraction, 0)

In [ ]:
for i in range(len(matrix[0, :])):
    for j in range(len(matrix[:, 0])):
        matrix[j, i] = max(matrix[j, i] - abs(noise_fft[i]),0)

In [ ]:
import seaborn as sns
plt.imshow(np.log10(matrix), aspect='auto', interpolation='none',
 extent = extents(t) + extents(f), origin='lower')

plt.title("Mix sound spectogram")
plt.xlabel('t (s)')
plt.ylabel('f (Hz)')
plt.show()

We can notice a significant difference in the matrix.

We can see some white regions in the spectogram. These correspond to frequency regions with amplitudes equal to or less than the amplitudes of the noise frequencies. With subtraction, these regions ended up going to zero.

Finally, we must understand how this noise signal correction affected the original signal.

The itfct() function will be applied to the resulting matrix of the correction, using the same values of nWin and nHop used in tfct()

In [ ]:
# Initializate parameters
x_length = len(x_array)
N_win = 7500
N_hop = round(N_win/2)

t_y, y_array = itfct(matrix, N_hop, N_win, 16000)

let's plot this signal obtained by the inverse stft.

In [ ]:
plt.plot(t_y, y_array)

plt.title("Signal reconstructed after noise correction")
plt.xlabel('t (s)')
plt.show()

Due to the same problem mentioned in the first part, unfortunately it is not possible to evaluate the use of the correction.